In [1]:
!pip install torchvision matplotlib numpy tensorboard standard-imghdr torchnet scikit-learn opencv-python Levenshtein


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [1]:
# imports
import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data.dataset import Dataset
from PIL import Image, ImageDraw, ImageFont
from sklearn.metrics import accuracy_score

from torch.utils.tensorboard import SummaryWriter

from ImageToStringNet import ImageToStringNet, classes
from ImageToStringNetDropout import ImageToStringNetDropout

import Levenshtein
import sys
sys.path.append('../core')
from main import process_image

# Evaluation

In [2]:
words_dir = "../dataset/english_words_dataset"
words_dataset = []
with open(f'{words_dir}/dataset.txt', encoding='utf-8') as f:
    for line in f:
        parts = line.rstrip('\n').split('\t', 1)
        if len(parts) == 2:
            words_dataset.append(parts)
        else:
            # gestisci eventuali righe malformate
            words_dataset.append([parts[0], ""])
#words_dataset = np.loadtxt(f'{words_dir}/dataset.txt', dtype=str, delimiter='\t', comments=[])

In [3]:
len(words_dataset)

1500

In [4]:
results = []
for i, data in enumerate(words_dataset):
	img_path = f'{words_dir}/{data[0]}'
	print("Processing: ", img_path)
	print(data[1])
	img = Image.open(img_path)
	try:
		_, output = process_image(img)
		edit_distance = Levenshtein.distance(data[1], output)

		results.append({
			'path': img_path,
			'label': data[1],
			'output': output,
			'edit_distance': edit_distance
		})
		print(f'{output}\nEDIT DISTANCE: {edit_distance}\n')
	except Exception as e:
		print(f'Error: {e}')
		results.append({
			'path': img_path,
			'label': data[1],
			'output': None,
			'edit_distance': None,
			'error': e
		})

Processing:  ../dataset/english_words_dataset/Lato-Medium/69cec03f-721d-4b85-b189-bd4d7e0c5ee3.png
I read a book one day and my whole life was changed.
l read a booK one day and mywhole life was changed.
EDIT DISTANCE: 3

Processing:  ../dataset/english_words_dataset/Lato-Medium/7b925ff7-7d09-4763-aa3f-3ffe61022e80.png
I will have you without armor, Kaz Brekker. Or I will not have you at all.
l will have you without armor, Kaz BreKKer. Or l will not have you at all.
EDIT DISTANCE: 4

Processing:  ../dataset/english_words_dataset/Lato-Medium/3a5391a0-6fc1-42f3-9b0d-906ecb1567f9.png
Imagination is like a muscle. I found out that the more I wrote, the bigger it got.
lmaginahon is liKe a muscle. l found out that the more l wrote, the bigger it got.
EDIT DISTANCE: 6

Processing:  ../dataset/english_words_dataset/Lato-Medium/dfb6ed90-554b-4742-90b8-a8caf8aa14c0.png
Anti-intellectualism has been a constant thread winding its way through our political and cultural life, nurtured by the false n

In [5]:
results[0]
edit_distances = [result['edit_distance'] for result in results if result['edit_distance'] is not None]
mean = np.mean(edit_distances)
variance = np.var(edit_distances)
minimum = np.min(edit_distances)
maximum = np.max(edit_distances)

print(f"Mean: {mean}, Variance: {variance}, Min: {minimum}, Max: {maximum}")

Mean: 48.339333333333336, Variance: 26490.061519555555, Min: 0, Max: 2188


In [6]:
max_edit_distance_objects = [result for result in results if result['edit_distance'] == maximum]
print(max_edit_distance_objects)

[{'path': '../dataset/english_words_dataset/Muli-SemiBold/14ad76ab-0344-44d9-871f-0348c73bcfa7.png', 'label': 'Once on a yellow piece of paper with green lines\the wrote a poemAnd he called it "Chops"\tbecause that was the name of his dogAnd that\'s what it was all aboutAnd his teacher gave him an A\tand a gold starAnd his mother hung it on the kitchen door\tand read it to his auntsThat was the year Father Tracy\ttook all the kids to the zooAnd he let them sing on the busAnd his little sister was born\twith tiny toenails and no hairAnd his mother and father kissed a lotAnd the girl around the corner sent him aValentine signed with a row of X\'s\tand he had to ask his father what the X\'s meantAnd his father always tucked him in bed at nightAnd was always there to do itOnce on a piece of white paper with blue lines\the wrote a poemAnd he called it "Autumn"\tbecause that was the name of the seasonAnd that\'s what it was all aboutAnd his teacher gave him an A\tand asked him to write more 

In [9]:
correct_predictions_case = sum(1 for result in results if result['output'] and result['label'] == result['output'])
correct_predictions = sum(1 for result in results if result['output'] and result['label'].lower() == result['output'].lower())
correct_predictions_no_spaces = sum(
	1 for result in results 
	if result['output'] and result['label'].replace(" ", "").lower() == result['output'].replace(" ", "").lower()
)
correct_predictions_no_spaces_no_Il = sum(
	1 for result in results
	if result['output'] and result['label'].replace(" ", "").replace("I", "l").lower() == result['output'].replace(" ", "").replace("I", "l").lower()
)
correct_total = sum(
	1 for result in results
	if True
)

accuracy_case = correct_predictions_case / len(results) * 100
accuracy = correct_predictions / len(results) * 100
accuracy_no_spaces = correct_predictions_no_spaces / len(results) * 100
accuracy_no_spaces_no_Il = correct_predictions_no_spaces_no_Il / len(results) * 100
accuracy_total = correct_total / len(results) * 100

print(f"Accuracy (case sensitive): {accuracy_case:.2f}%")
print(f"Accuracy (case insensitive): {accuracy:.2f}%")
print(f"Accuracy (case insensitive, no spaces inside): {accuracy_no_spaces:.2f}%")
print(f"Accuracy (case insensitive, no spaces inside, no Il diff): {accuracy_no_spaces_no_Il:.2f}%")
print(f"Accuracy (case insensitive, no spaces inside, no Il diff, no anything): {accuracy_total:.2f}%")


Accuracy (case sensitive): 7.53%
Accuracy (case insensitive): 15.00%
Accuracy (case insensitive, no spaces inside): 19.00%
Accuracy (case insensitive, no spaces inside, no Il diff): 36.87%
Accuracy (case insensitive, no spaces inside, no Il diff, no anything): 100.00%


In [10]:
for result in results:
    if result['edit_distance'] is not None and len(result['label']) > 0:
        result['edit_distance_pct'] = (result['edit_distance'] / len(result['label'])) * 100
    else:
        result['edit_distance_pct'] = None

In [11]:
# Calcola la media delle percentuali di edit distance (edit_distance_pct) ignorando i None
edit_distance_pcts = [result['edit_distance_pct'] for result in results if result['edit_distance_pct'] is not None]
edit_distance_pct_avg = np.mean(edit_distance_pcts)
print(f"Average edit distance percentage: {edit_distance_pct_avg:.2f}%")

Average edit distance percentage: 19.20%


In [12]:
total_edit_distance = sum(result['edit_distance'] for result in results if result['edit_distance'] is not None)
total_label_length = sum(len(result['label']) for result in results if result['edit_distance'] is not None)
average_edit_distance_per_char = total_edit_distance / total_label_length if total_label_length > 0 else None
print(f"Average edit distance per character: {average_edit_distance_per_char*100:.2f}%")

Average edit distance per character: 32.26%


In [13]:
# Stampa i risultati con edit_distance diverso da 0
nonzero_edit_distance = [result for result in results if result['output'] and result['label'] != result['output']]
for r in nonzero_edit_distance:
    print(r)

{'path': '../dataset/english_words_dataset/Lato-Medium/69cec03f-721d-4b85-b189-bd4d7e0c5ee3.png', 'label': 'I read a book one day and my whole life was changed.', 'output': 'l read a booK one day and mywhole life was changed.', 'edit_distance': 3, 'edit_distance_pct': 5.769230769230769}
{'path': '../dataset/english_words_dataset/Lato-Medium/7b925ff7-7d09-4763-aa3f-3ffe61022e80.png', 'label': 'I will have you without armor, Kaz Brekker. Or I will not have you at all.', 'output': 'l will have you without armor, Kaz BreKKer. Or l will not have you at all.', 'edit_distance': 4, 'edit_distance_pct': 5.405405405405405}
{'path': '../dataset/english_words_dataset/Lato-Medium/3a5391a0-6fc1-42f3-9b0d-906ecb1567f9.png', 'label': 'Imagination is like a muscle. I found out that the more I wrote, the bigger it got.', 'output': 'lmaginahon is liKe a muscle. l found out that the more l wrote, the bigger it got.', 'edit_distance': 6, 'edit_distance_pct': 7.228915662650602}
{'path': '../dataset/english_